In [53]:
import gym
import numpy as np
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

import tensorflow as tf
import collections
import datetime as dt
from tensorboard import summary as summary_lib

env = gym.make('CartPole-v1')

np.random.seed(1)


class ValueEstimator():
    """
    Value Function approximator.
    """

    def __init__(self, state_size, scope="value_estimator"):
        with tf.variable_scope(scope):
            self.state = tf.placeholder(tf.float32, [1, state_size], "state")
            self.target = tf.placeholder(dtype=tf.float32, name="target")
            self.learning_rate = tf.placeholder(dtype=tf.float32, name="learning_rate")

            # This is just table lookup estimator
#             state_one_hot = tf.one_hot(, int(state_size))
#             print(state_one_hot)
            self.input_layer = tf.layers.dense(
                inputs=self.state,
                units=32,
                activation='relu',
                kernel_initializer=tf.contrib.layers.xavier_initializer(seed=0))
#             self.hidden_layer = tf.layers.dense(
#                 inputs = self.input_layer,
#                 units=16,
#                 activation='relu',
#                 kernel_initializer=tf.contrib.layers.xavier_initializer(seed=0))
            self.output_layer = tf.layers.dense(
                inputs=self.input_layer,
                units=1,
                activation=None,
                kernel_initializer=tf.contrib.layers.xavier_initializer(seed=0))
            self.value_estimate = tf.squeeze(self.output_layer)
            self.loss = tf.reduce_sum(tf.square(self.target - self.value_estimate))
#             self.loss = tf.squared_difference(self.value_estimate, self.target)
#             self.loss = tf.losses.mean_squared_error(self.value_estimate, self.target)

            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            self.train_op = self.optimizer.minimize(
                self.loss, global_step=tf.contrib.framework.get_global_step())

    def predict(self, state, sess=None):
#         state = state.reshape(-1)
        sess = sess or tf.get_default_session()
        return sess.run(self.value_estimate, {self.state: state})

    def update(self, state, target, learning_rate, sess=None):
#         state = state.reshape(-1)
        sess = sess or tf.get_default_session()
        feed_dict = {self.state: state, self.target: target, self.learning_rate:learning_rate}
        _, loss = sess.run([self.train_op, self.loss], feed_dict)
        return loss

class PolicyNetwork:
    def __init__(self, state_size, action_size, name='policy_network'):
        self.state_size = state_size
        self.action_size = action_size

        with tf.variable_scope(name):
            self.learning_rate = tf.placeholder(dtype=tf.float32, name="learning_rate")
            self.state = tf.placeholder(tf.float32, [None, self.state_size], name="state")
            self.action = tf.placeholder(tf.int32, [self.action_size], name="action")
            self.R_t = tf.placeholder(tf.float32, name="total_rewards")

            self.W1 = tf.get_variable("W1", [self.state_size, 12],
                                      initializer=tf.contrib.layers.xavier_initializer(seed=0))
            self.b1 = tf.get_variable("b1", [12], initializer=tf.zeros_initializer())
            self.W2 = tf.get_variable("W2", [12, self.action_size],
                                      initializer=tf.contrib.layers.xavier_initializer(seed=0))
            self.b2 = tf.get_variable("b2", [self.action_size], initializer=tf.zeros_initializer())

            self.Z1 = tf.add(tf.matmul(self.state, self.W1), self.b1)
            self.A1 = tf.nn.relu(self.Z1)
            self.output = tf.add(tf.matmul(self.A1, self.W2), self.b2)

            # Softmax probability distribution over actions
            self.actions_distribution = tf.squeeze(tf.nn.softmax(self.output))
            # Loss with negative log probability
            self.neg_log_prob = tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.output, labels=self.action)
            self.loss = tf.reduce_mean(self.neg_log_prob * self.R_t)
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate).minimize(self.loss)
    
# Define hyperparameters
state_size = 4
action_size = env.action_space.n

max_episodes = 5000
max_steps = 501
discount_factor = 0.99
learning_rate = 0.001
value_learning_rate = 0.005
render = False

# Initialize the policy network
tf.reset_default_graph()
policy = PolicyNetwork(state_size, action_size)
value_estimator = ValueEstimator(state_size)

LOGDIR = './TensorBoard/Q2' + f"/DQLearning_{dt.datetime.now().strftime('%d%m%Y%H%M')}"
# Start training the agent with REINFORCE algorithm
with tf.Session() as sess, tf.summary.FileWriter(LOGDIR) as tb_logger:
    sess.run(tf.global_variables_initializer())
    solved = False
    episode_rewards = np.zeros(max_episodes)
    average_rewards = 0.0
    step_done = 0

    sliding_avg = collections.deque(maxlen=100)
    avg_loss = 0.0
    for episode in range(max_episodes):
        if episode % 300 == 0 and episode > 0:
            learning_rate*=0.1
            value_learning_rate*=0.1
            print(learning_rate, value_learning_rate)
            
        state = env.reset()
        state = state.reshape([1, state_size])
        episode_transitions = []
        I = 1
        for step in range(max_steps):

            actions_distribution = sess.run(policy.actions_distribution, {policy.state: state})
            action = np.random.choice(np.arange(len(actions_distribution)), p=actions_distribution)
            next_state, reward, done, _ = env.step(action)
            
            next_state = next_state.reshape([1, state_size])
            episode_rewards[episode] += reward
            reward = +1 if not done or step == 500 else -10
            if render:
                env.render()
            value_s = value_estimator.predict(state)
            value_next = 0
            if done:
                delta = reward - value_s
            else:
                value_next = value_estimator.predict(next_state)
                delta = reward + (discount_factor*value_next) - value_s
            
            total_discounted_return = I*delta
            value_estimator.update(state, reward + discount_factor*value_next, value_learning_rate)
            
            action_one_hot = np.zeros(action_size)
            action_one_hot[action] = 1
        
            feed_dict = {policy.state: state, policy.R_t: total_discounted_return,
                         policy.action: action_one_hot, policy.learning_rate:learning_rate}
            _, loss = sess.run([policy.optimizer, policy.loss], feed_dict)
#             if loss < -150:
#                 print(step, total_discounted_return, reward, value_s, value_next)
            avg_loss += loss

            if done:
                step_done = step + 1
                sliding_avg.append(step_done)
                if episode > 98:
                    # Check if solved
                    average_rewards = np.mean(episode_rewards[(episode - 99):episode + 1])
                print("Episode {} Reward: {} Average over 100 episodes: {}".format(episode, episode_rewards[episode],
                                                                                   round(average_rewards, 2)))
                if average_rewards > 475:
                    print(' Solved at episode: ' + str(episode))
                    solved = True
                break
                
#             I = discount_factor*I
            state = next_state

        if solved:
            break
            
        summary = tf.Summary(value=[tf.Summary.Value(tag='reward',
                                                     simple_value=step_done),
                                   tf.Summary.Value(tag='avg_loss',
                                                     simple_value=avg_loss / step_done),
                                   tf.Summary.Value(tag='reward_avg_100_eps',
                                                     simple_value=sum(sliding_avg) / len(sliding_avg))])
        tb_logger.add_summary(summary, episode)


Episode 0 Reward: 12.0 Average over 100 episodes: 0.0
Episode 1 Reward: 32.0 Average over 100 episodes: 0.0
Episode 2 Reward: 12.0 Average over 100 episodes: 0.0
Episode 3 Reward: 18.0 Average over 100 episodes: 0.0
Episode 4 Reward: 16.0 Average over 100 episodes: 0.0
Episode 5 Reward: 45.0 Average over 100 episodes: 0.0
Episode 6 Reward: 26.0 Average over 100 episodes: 0.0
Episode 7 Reward: 18.0 Average over 100 episodes: 0.0
Episode 8 Reward: 30.0 Average over 100 episodes: 0.0
Episode 9 Reward: 33.0 Average over 100 episodes: 0.0
Episode 10 Reward: 17.0 Average over 100 episodes: 0.0
Episode 11 Reward: 10.0 Average over 100 episodes: 0.0
Episode 12 Reward: 24.0 Average over 100 episodes: 0.0
Episode 13 Reward: 9.0 Average over 100 episodes: 0.0
Episode 14 Reward: 36.0 Average over 100 episodes: 0.0
Episode 15 Reward: 9.0 Average over 100 episodes: 0.0
Episode 16 Reward: 15.0 Average over 100 episodes: 0.0
Episode 17 Reward: 10.0 Average over 100 episodes: 0.0
Episode 18 Reward: 21.

Episode 147 Reward: 335.0 Average over 100 episodes: 78.45
Episode 148 Reward: 116.0 Average over 100 episodes: 79.39
Episode 149 Reward: 165.0 Average over 100 episodes: 80.37
Episode 150 Reward: 81.0 Average over 100 episodes: 80.71
Episode 151 Reward: 77.0 Average over 100 episodes: 81.14
Episode 152 Reward: 80.0 Average over 100 episodes: 81.65
Episode 153 Reward: 28.0 Average over 100 episodes: 81.77
Episode 154 Reward: 65.0 Average over 100 episodes: 82.28
Episode 155 Reward: 24.0 Average over 100 episodes: 82.09
Episode 156 Reward: 77.0 Average over 100 episodes: 82.22
Episode 157 Reward: 74.0 Average over 100 episodes: 82.85
Episode 158 Reward: 30.0 Average over 100 episodes: 82.81
Episode 159 Reward: 46.0 Average over 100 episodes: 82.5
Episode 160 Reward: 117.0 Average over 100 episodes: 83.42
Episode 161 Reward: 69.0 Average over 100 episodes: 83.73
Episode 162 Reward: 62.0 Average over 100 episodes: 84.23
Episode 163 Reward: 134.0 Average over 100 episodes: 85.18
Episode 16

Episode 285 Reward: 192.0 Average over 100 episodes: 315.09
Episode 286 Reward: 213.0 Average over 100 episodes: 314.17
Episode 287 Reward: 236.0 Average over 100 episodes: 314.71
Episode 288 Reward: 231.0 Average over 100 episodes: 315.66
Episode 289 Reward: 185.0 Average over 100 episodes: 315.94
Episode 290 Reward: 191.0 Average over 100 episodes: 316.35
Episode 291 Reward: 215.0 Average over 100 episodes: 313.8
Episode 292 Reward: 205.0 Average over 100 episodes: 314.07
Episode 293 Reward: 207.0 Average over 100 episodes: 314.37
Episode 294 Reward: 210.0 Average over 100 episodes: 311.47
Episode 295 Reward: 230.0 Average over 100 episodes: 309.87
Episode 296 Reward: 252.0 Average over 100 episodes: 307.39
Episode 297 Reward: 244.0 Average over 100 episodes: 304.83
Episode 298 Reward: 187.0 Average over 100 episodes: 301.7
Episode 299 Reward: 178.0 Average over 100 episodes: 300.88
0.0001 0.0005
Episode 300 Reward: 184.0 Average over 100 episodes: 297.72
Episode 301 Reward: 194.0 Av

Episode 422 Reward: 500.0 Average over 100 episodes: 424.78
Episode 423 Reward: 500.0 Average over 100 episodes: 427.6
Episode 424 Reward: 500.0 Average over 100 episodes: 430.38
Episode 425 Reward: 500.0 Average over 100 episodes: 432.93
Episode 426 Reward: 500.0 Average over 100 episodes: 435.43
Episode 427 Reward: 500.0 Average over 100 episodes: 438.14
Episode 428 Reward: 500.0 Average over 100 episodes: 440.59
Episode 429 Reward: 500.0 Average over 100 episodes: 443.26
Episode 430 Reward: 500.0 Average over 100 episodes: 445.83
Episode 431 Reward: 500.0 Average over 100 episodes: 448.3
Episode 432 Reward: 500.0 Average over 100 episodes: 450.73
Episode 433 Reward: 500.0 Average over 100 episodes: 453.22
Episode 434 Reward: 500.0 Average over 100 episodes: 455.41
Episode 435 Reward: 500.0 Average over 100 episodes: 457.57
Episode 436 Reward: 500.0 Average over 100 episodes: 459.71
Episode 437 Reward: 500.0 Average over 100 episodes: 461.78
Episode 438 Reward: 500.0 Average over 100

In [3]:
env.close()